In [9]:
%cd Cardsformer

/home/s2430014/research/enhance-cardsformer-dev/Cardsformer


/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [11]:
import os

# 環境変数を設定
os.environ["PYTHONNET_RUNTIME"] = "coreclr"

# 設定した環境変数を取得
print(os.environ["PYTHONNET_RUNTIME"])

coreclr


In [12]:
from Env.EnvWrapper import Environment
from Env.Hearthstone import Hearthstone

In [223]:
from experiment.util.model_util import load_prediction_model, load_policy_model, load_encoder
from experiment.util.data_util  import EvalPredictionDataset

In [25]:
import torch

In [30]:
from Model.ModelWrapper import Model

In [33]:
import numpy as np

In [134]:
from Algo.utils import create_buffers

4715
2
max_model is prediction_model4715.tar


In [248]:
from Algo.utils import get_batch

In [221]:
from experiment.util.data_util import NpyLogData

In [ ]:
from 

In [13]:
positions = ['Player1', 'Player2']

In [14]:
from torch import multiprocessing as mp
ctx = mp.get_context('spawn')

In [116]:
class Flags:
    def __init__(self):
        self.exp_epsilon = 0.3
        self.unroll_length = 100
        self.num_buffers = 30
    
flags = Flags()

In [118]:
def simulate_game():
    while True:
        num_options = len(options)
        if num_options == 1:
            action = options[0]
        else:
            hand_card_embed = encoder.encode(obs['hand_card_names'])
            minion_embed = encoder.encode(obs['minion_names'])
            weapon_embed = encoder.encode(obs['weapon_names'])
            secret_embed = encoder.encode(obs['secret_names'])
            with torch.no_grad():
                next_state = prediction_model([hand_card_embed, minion_embed, weapon_embed, obs['hand_card_scalar_batch'], obs['minion_scalar_batch'], obs['hero_scalar_batch']])
            obs['next_minion_scalar'] = next_state[0]
            obs['next_hero_scalar'] = next_state[1]
            with torch.no_grad():
                agent_output = model.forward(hand_card_embed, minion_embed, secret_embed, weapon_embed, obs, num_options, actor = True)
            agent_output = agent_output.argmax()
            if np.random.rand() < flags.exp_epsilon:
                _action_idx = torch.randint(len(options), (1, ))[0].item()
            else:
                _action_idx = int(agent_output.cpu().detach().numpy())
            
            assert isinstance(_action_idx, int), f"_action_idx の型が int ではありません type: {type(_action_idx)}, _action_idx: {_action_idx}"
            action = options[_action_idx]
            hand_card_embed_buf[position].append(hand_card_embed)
            minion_embed_buf[position].append(minion_embed)
            weapon_embed_buf[position].append(weapon_embed)
            secret_embed_buf[position].append(secret_embed)
            hand_card_scalar_buf[position].append(obs["hand_card_scalar_batch"][_action_idx])
            minion_scalar_buf[position].append(obs["minion_scalar_batch"][_action_idx])
            hero_scalar_buf[position].append(obs["hero_scalar_batch"][_action_idx])
            next_minion_scalar_buf[position].append(obs['next_minion_scalar'][_action_idx])
            next_hero_scalar_buf[position].append(obs['next_hero_scalar'][_action_idx])
            # save key info buf here
            size[position] += 1
        
        position, obs, options, done, episode_return, _ = env.step(action)
        if done:
            for p in positions:
                diff = size[p] - len(target_buf[p])
                if diff > 0:
                    done_buf[p].extend([False for _ in range(diff-1)])
                    done_buf[p].append(True)
                    episode_return = episode_return if p == 'Player1' else -episode_return
                    episode_return_buf[p].extend([0.0 for _ in range(diff-1)])
                    episode_return_buf[p].append(episode_return)
                    target_buf[p].extend([episode_return for _ in range(diff)])
            break
        # 一試合終了

    return 

In [119]:
def write_in_buffer():
    for p in positions:
        while size[p] > T: 
            index = free_queue[p].get()
            if index is None:
                break
            for t in range(T):
                buffers[p]['done'][index][t, ...] = done_buf[p][t]
                buffers[p]['episode_return'][index][t, ...] = episode_return_buf[p][t]
                buffers[p]['target'][index][t, ...] = target_buf[p][t]
                buffers[p]['hand_card_embed'][index][t, ...] = hand_card_embed_buf[p][t]
                buffers[p]['minion_embed'][index][t, ...] = minion_embed_buf[p][t]
                buffers[p]['weapon_embed'][index][t, ...] = weapon_embed_buf[p][t]
                buffers[p]['secret_embed'][index][t, ...] = secret_embed_buf[p][t]
                buffers[p]['hand_card_scalar'][index][t, ...] =	hand_card_scalar_buf[p][t] #次元を変更したことによるエラー
                buffers[p]['minion_scalar'][index][t, ...] = minion_scalar_buf[p][t]
                buffers[p]['hero_scalar'][index][t, ...] = hero_scalar_buf[p][t]
                buffers[p]['next_minion_scalar'][index][t, ...] = next_minion_scalar_buf[p][t]
                buffers[p]['next_hero_scalar'][index][t, ...] = next_hero_scalar_buf[p][t]
            full_queue[p].put(index)
            done_buf[p] = done_buf[p][T:]
            episode_return_buf[p] = episode_return_buf[p][T:]
            hand_card_embed_buf[p] = hand_card_embed_buf[p][T:]
            minion_embed_buf[p] = minion_embed_buf[p][T:]
            weapon_embed_buf[p] = weapon_embed_buf[p][T:]
            secret_embed_buf[p] = secret_embed_buf[p][T:]
            hand_card_scalar_buf[p] = hand_card_scalar_buf[p][T:]
            minion_scalar_buf[p] = minion_scalar_buf[p][T:]
            hero_scalar_buf[p] = hero_scalar_buf[p][T:]
            target_buf[p] = target_buf[p][T:]
            next_minion_scalar_buf[p] = next_minion_scalar_buf[p][T:]
            next_hero_scalar_buf[p] = next_hero_scalar_buf[p][T:]
            
            size[p] -= T

In [170]:
device = "cpu"

In [173]:
# modelに渡されるのはなに？ ->いったんpolicy modelわたしとくか
model = Model(device=device)

In [174]:
# queueはただの辞書？

buffers = create_buffers(flags, device_iterator)
buffers = buffers["cpu"]

device_iterator = ['cpu']

free_queue = {}
full_queue = {}
for device in device_iterator:
    _free_queue = {
        'Player1': ctx.SimpleQueue(),
        'Player2': ctx.SimpleQueue()
    }
    _full_queue = {
        'Player1': ctx.SimpleQueue(),
        'Player2': ctx.SimpleQueue()
    }
    free_queue[device] = _free_queue
    full_queue[device] = _full_queue

for device in device_iterator:
    for m in range(flags.num_buffers):
        free_queue[device]['Player1'].put(m)
        free_queue[device]['Player2'].put(m)

# 実際各プロセスに入力されるqueueは以下の値
free_queue = free_queue["cpu"]
full_queue = full_queue["cpu"]

game = Hearthstone()
env = Environment(game, device)

T = flags.unroll_length

# initialize buffers
done_buf = {p: [] for p in positions}
episode_return_buf = {p: [] for p in positions}
target_buf = {p: [] for p in positions}
hand_card_embed_buf = {p: [] for p in positions}
minion_embed_buf = {p: [] for p in positions}
weapon_embed_buf = {p: [] for p in positions}
secret_embed_buf = {p: [] for p in positions}
hand_card_scalar_buf = {p: [] for p in positions}
minion_scalar_buf = {p: [] for p in positions}
hero_scalar_buf = {p: [] for p in positions}
next_minion_scalar_buf = {p: [] for p in positions}
next_hero_scalar_buf = {p: [] for p in positions}
size = {p: 0 for p in positions}

# load models util関数で共通化できそう
# tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")
# auto_model = AutoModel.from_pretrained("sentence-transformers/all-mpnet-base-v2")
# encoder = Encoder(model=auto_model, tokenizer=tokenizer)
# encoder.to(device)
# position, obs, options, done, episode_return = env.initial()
# prediction_model = PredictionModel()
# checkpoint_states = torch.load(f"./trained_models/prediction_model4715.tar", map_location='cpu')['model_state_dict'] 
# new_state_dict = typing.OrderedDict()
# for k, v in checkpoint_states.items():
#     name = k[7:]
#     new_state_dict[name] = v

# prediction_model.load_state_dict(new_state_dict)
# prediction_model.to(device)
# prediction_model.eval()

encoder = load_encoder()
prediction_model = load_prediction_model("./trained_models/prediction_model4715.tar")
position, obs, options, done, episode_return = env.initial()

# while True: #永久にroopするためのwhileなので外す
# ゲームシミュレーションがスタート
for i in range(100):
    while True:
        num_options = len(options)
        if num_options == 1:
            action = options[0]
        else:
            hand_card_embed = encoder.encode(obs['hand_card_names'])
            minion_embed = encoder.encode(obs['minion_names'])
            weapon_embed = encoder.encode(obs['weapon_names'])
            secret_embed = encoder.encode(obs['secret_names'])
            with torch.no_grad():
                next_state = prediction_model([hand_card_embed, minion_embed, weapon_embed, obs['hand_card_scalar_batch'], obs['minion_scalar_batch'], obs['hero_scalar_batch']])
            obs['next_minion_scalar'] = next_state[0]
            obs['next_hero_scalar'] = next_state[1]
            with torch.no_grad():
                agent_output = model.forward(hand_card_embed, minion_embed, secret_embed, weapon_embed, obs, num_options, actor = True)
            agent_output = agent_output.argmax()
            if np.random.rand() < flags.exp_epsilon:
                _action_idx = torch.randint(len(options), (1, ))[0].item()
            else:
                _action_idx = int(agent_output.cpu().detach().numpy())
            
            assert isinstance(_action_idx, int), f"_action_idx の型が int ではありません type: {type(_action_idx)}, _action_idx: {_action_idx}"
            action = options[_action_idx]
            hand_card_embed_buf[position].append(hand_card_embed)
            minion_embed_buf[position].append(minion_embed)
            weapon_embed_buf[position].append(weapon_embed)
            secret_embed_buf[position].append(secret_embed)
            hand_card_scalar_buf[position].append(obs["hand_card_scalar_batch"][_action_idx])
            minion_scalar_buf[position].append(obs["minion_scalar_batch"][_action_idx])
            hero_scalar_buf[position].append(obs["hero_scalar_batch"][_action_idx])
            next_minion_scalar_buf[position].append(obs['next_minion_scalar'][_action_idx])
            next_hero_scalar_buf[position].append(obs['next_hero_scalar'][_action_idx])
            # save key info buf here
            size[position] += 1
        
        position, obs, options, done, episode_return, _ = env.step(action)
        if done:
            for p in positions:
                diff = size[p] - len(target_buf[p])
                if diff > 0:
                    done_buf[p].extend([False for _ in range(diff-1)])
                    done_buf[p].append(True)
                    episode_return = episode_return if p == 'Player1' else -episode_return
                    episode_return_buf[p].extend([0.0 for _ in range(diff-1)])
                    episode_return_buf[p].append(episode_return)
                    target_buf[p].extend([episode_return for _ in range(diff)])
            print("the game ends")
            break
            
        # 一試合終了
    
    for p in positions:
        while size[p] > T: 
            index = free_queue[p].get()
            print(index)
            if index is None:
                break
            for t in range(T):
                buffers[p]['done'][index][t, ...] = done_buf[p][t]
                buffers[p]['episode_return'][index][t, ...] = episode_return_buf[p][t]
                buffers[p]['target'][index][t, ...] = target_buf[p][t]
                buffers[p]['hand_card_embed'][index][t, ...] = hand_card_embed_buf[p][t]
                buffers[p]['minion_embed'][index][t, ...] = minion_embed_buf[p][t]
                buffers[p]['weapon_embed'][index][t, ...] = weapon_embed_buf[p][t]
                buffers[p]['secret_embed'][index][t, ...] = secret_embed_buf[p][t]
                buffers[p]['hand_card_scalar'][index][t, ...] =	hand_card_scalar_buf[p][t] #次元を変更したことによるエラー
                buffers[p]['minion_scalar'][index][t, ...] = minion_scalar_buf[p][t]
                buffers[p]['hero_scalar'][index][t, ...] = hero_scalar_buf[p][t]
                buffers[p]['next_minion_scalar'][index][t, ...] = next_minion_scalar_buf[p][t]
                buffers[p]['next_hero_scalar'][index][t, ...] = next_hero_scalar_buf[p][t]
            full_queue[p].put(index)
            done_buf[p] = done_buf[p][T:]
            episode_return_buf[p] = episode_return_buf[p][T:]
            hand_card_embed_buf[p] = hand_card_embed_buf[p][T:]
            minion_embed_buf[p] = minion_embed_buf[p][T:]
            weapon_embed_buf[p] = weapon_embed_buf[p][T:]
            secret_embed_buf[p] = secret_embed_buf[p][T:]
            hand_card_scalar_buf[p] = hand_card_scalar_buf[p][T:]
            minion_scalar_buf[p] = minion_scalar_buf[p][T:]
            hero_scalar_buf[p] = hero_scalar_buf[p][T:]
            target_buf[p] = target_buf[p][T:]
            next_minion_scalar_buf[p] = next_minion_scalar_buf[p][T:]
            next_hero_scalar_buf[p] = next_hero_scalar_buf[p][T:]
            
            size[p] -= T

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


the game ends
the game ends
the game ends
0
0
the game ends
1
1
the game ends
the game ends
2
the game ends
2
the game ends
3
the game ends
3
the game ends
4
the game ends
4
the game ends
5
the game ends
5
the game ends
6
the game ends
7
6
the game ends
7
the game ends
the game ends
8
the game ends
8
the game ends
9
the game ends
10
9
the game ends
the game ends
10
the game ends
11
11
the game ends
the game ends
12
the game ends
13
12
the game ends
14
the game ends
13
the game ends
14
the game ends
15
15
the game ends
the game ends
16
16
the game ends
the game ends
17
the game ends
17
18
the game ends
18
19
the game ends
the game ends
19
20
the game ends
the game ends
21
the game ends
20
the game ends
22
the game ends
21
the game ends
23
the game ends
22
24
the game ends
the game ends
23
25
the game ends
the game ends
24
the game ends
25
26
the game ends
27
the game ends
26
the game ends
28
the game ends
27
the game ends
29
the game ends
28
the game ends


KeyboardInterrupt: 

In [175]:
free_queue["Player1"].empty()

False

In [176]:
full_queue["Player1"].empty()

False

In [142]:
free_queue["Player1"].get()

1

In [189]:
full_queue["Player1"].get()

9

In [177]:
len(buffers["Player1"]["done"])

30

In [207]:
len(buffers["Player1"]["episode_return"])

30

In [208]:
buffers["Player1"]["episode_return"][0].shape

torch.Size([100])

In [209]:
buffers["Player1"]["episode_return"][0]

tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.])

In [216]:
buffers_2 = create_buffers(flags, device_iterator)
buffers_2 = buffers_2["cpu"]

In [220]:
buffers_2["Player1"].keys()

dict_keys(['done', 'episode_return', 'target', 'hand_card_embed', 'minion_embed', 'weapon_embed', 'secret_embed', 'hand_card_scalar', 'minion_scalar', 'hero_scalar', 'next_minion_scalar', 'next_hero_scalar'])

In [222]:
npy_log = NpyLogData()

In [224]:
ds = EvalPredictionDataset(["./off_line_data_vs_ai0.npy"])

=== load prediction dataset:./off_line_data_vs_ai0.npy ===


In [225]:
ds

In [270]:
ds.data.keys()

dict_keys(['hand_card_names', 'minion_names', 'weapon_names', 'hand_card_scalar', 'minion_scalar', 'hero_scalar', 'next_state_minion_scalar', 'next_state_hero_scalar'])

In [243]:
tr = [i.numpy() for i in  buffers_2["Player1"]["hand_card_embed"][0]]

In [247]:
tr[0].shape

(11, 768)

In [263]:
d_tmp = buffers["Player1"]

In [264]:
torch.save(d_tmp, "tmp.pth")

In [265]:
loaded_data = torch.load("tmp.pth")

In [267]:
def tranform_batch_to_npy(batch):
    tr = [i.numpy() for i in  buffers_2["Player1"]["hand_card_embed"]]


SyntaxError: incomplete input (3845685511.py, line 1)

In [281]:
buffers_2["Player1"]["next_minion_scalar"][0].shape

torch.Size([100, 14, 9])

In [283]:
len(ds.data["hand_card_scalar"])

66720

In [282]:
ds